In [2]:
import sys
path = ".."
if path not in sys.path:
    sys.path.insert(0, path)

In [3]:
from data_retrieval import lipade_groundtruth
from transformers import AutoProcessor, Blip2ForConditionalGeneration
import matplotlib.pyplot as plt
from absolute_path import absolutePath
from PIL import Image
from tqdm import tqdm
import torch
import os

corpus = "lipade_groundtruth"

processor = AutoProcessor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16)

device = "cpu" # "cuda" if torch.cuda.is_available() else "cpu"
model.to(device).half()

/home/samuel-g/.pyenv/versions/3.11.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.76s/it]


Blip2ForConditionalGeneration(
  (vision_model): Blip2VisionModel(
    (embeddings): Blip2VisionEmbeddings(
      (patch_embedding): Conv2d(3, 1408, kernel_size=(14, 14), stride=(14, 14))
    )
    (encoder): Blip2Encoder(
      (layers): ModuleList(
        (0-38): 39 x Blip2EncoderLayer(
          (self_attn): Blip2Attention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=1408, out_features=4224, bias=True)
            (projection): Linear(in_features=1408, out_features=1408, bias=True)
          )
          (layer_norm1): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
          (mlp): Blip2MLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=1408, out_features=6144, bias=True)
            (fc2): Linear(in_features=6144, out_features=1408, bias=True)
          )
          (layer_norm2): LayerNorm((1408,), eps=1e-06, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((

In [4]:
x,_,y = lipade_groundtruth.getDataset(mode="similar")
images = []
for i in range(len(x)):
    images.append(Image.open(x[i]).convert('RGB'))

In [5]:
path = absolutePath + 'data_generation/generated/' + corpus + "/"
l = len(os.listdir(path))

for i in tqdm(range(len(x[l:]))):
    image = images[l + i]
    filename = x[l + i].split('/')[-1].split('.')[0] + '.txt'

    inputs = processor(image, return_tensors="pt").to(device, torch.float16)
    generated_ids = model.generate(**inputs, max_new_tokens=30)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0].strip()
    
    with open(path + filename, 'w') as f:
        f.write(generated_text)

100%|██████████| 262/262 [49:16:17<00:00, 677.01s/it]    
